In [1]:
import ResearchTool
tool=ResearchTool.tool

In [2]:
result = tool.invoke({'url':'https://zapier.com/blog/deepseek-vs-chatgpt/','lang':'Hindi'})
print(result)

[{'Answer': 'निम्नलिखित लेख का संक्षेप:\n\nडीपीसीईक और चैटजीपीटी, दो आर्टिफिशियल इंटेलिजेंस टूल्स की तुलना करता है। डीपीसीईक एक विजुअल प्रोग्रामिंग टूल है जिसमें उपयोगकर्ता कस्टम चैटबॉट बना सकते हैं, जबकि चैटजीपीटी ओपनएआई द्वारा विकसित एक भाषा मॉडल है। लेख के दोनों टूल के लाभ और सीमाएं Discuss करता है, उनकी संभावित एप्लिकेशन्स, जिसमें कार्य प्रवाह को ऑटोमेट करना, कस्टम चैटबॉट बनाना और उत्पादकता में सुधार शामिल हैं। अंत में, लेख निष्कर्ष निकालता है कि उपयोगकर्ता की विशेष आवश्यकताओं और लक्ष्यों पर निर्भर करता है कि किस टूल का चयन किया जाना है।'}]


In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import SystemMessage, ToolMessage, HumanMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


class Agent:
    def __init__(self,model,tools,system=""):
        self.system=system
        graph=StateGraph(AgentState)
        graph.add_node("llm",self.call_groq)
        graph.add_node("Research_summarizer",self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.action_exists,
            {True:"Research_summarizer",False:END}
        )
        graph.add_edge("Research_summarizer","llm")
        graph.set_entry_point("llm")
        self.graph=graph.compile()
        self.tools={t.name:t for t in tools}
        self.model=model.bind_tools(tools)
        
    def action_exists(self,state:AgentState):
        result=state['messages'][-1]
        return len(result.tool_calls)>0
        
        
    def call_groq(self,state:AgentState):
        messages=state["messages"]
        if self.system:
            messages=[SystemMessage(content=self.system)] + messages
        message=self.model.invoke(messages)
        return {'messages':[message]}
    
    def take_action(self,state:AgentState):
        tool_calls=state['messages'][-1].tool_calls
        results=[]
        for t in tool_calls:
            print(f'Calling: {t}')
            result=self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t["id"],name=t["name"],content=str(result)))
        print("Back to the Model !!")
        return {"messages":results}
    

In [4]:
from langchain_groq import ChatGroq
prompt = """You are a smart research assistant. Use the research_summarizer tool to help answer user questions.
You are allowed to make multiple calls (either together or in sequence)."""

tools=[tool]
model=ChatGroq(model='llama3-8b-8192')
abot=Agent(model,tools,prompt)

In [5]:
print(abot.graph.get_graph().draw_ascii())

                 +-----------+             
                 | __start__ |             
                 +-----------+             
                        *                  
                        *                  
                        *                  
                    +-----+                
                    | llm |.               
                    +-----+ .              
                 ***         ...           
                *               .          
              **                 ..        
+---------------------+       +---------+  
| Research_summarizer |       | __end__ |  
+---------------------+       +---------+  


In [6]:
messages=[HumanMessage(content="summarize the text on the webpage https://zapier.com/blog/deepseek-vs-chatgpt/ in Hindi")]
for event in abot.graph.stream({'messages':messages}):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_syem', 'function': {'arguments': '{"lang":"hi","url":"https://zapier.com/blog/deepseek-vs-chatgpt/"}', 'name': 'Research Summarizer'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 1025, 'total_tokens': 1125, 'completion_time': 0.083333333, 'prompt_time': 0.142035028, 'queue_time': 0.244627158, 'total_time': 0.225368361}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c5604e2d-eead-4854-b5ca-3855edb0272f-0', tool_calls=[{'name': 'Research Summarizer', 'args': {'lang': 'hi', 'url': 'https://zapier.com/blog/deepseek-vs-chatgpt/'}, 'id': 'call_syem', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1025, 'output_tokens': 100, 'total_tokens': 1125})]}
Calling: {'name': 'Research Summarizer', 'args': {'lang': 'hi', 'url': 'https://zapier.com/blog/deepseek-vs-chatgpt/'}

In [11]:
for x in result['messages'][-1].content.split('.'):
    print(x.strip())

डीपसीक और चैटजीपीटी की तुलना

डीपसीक और चैटजीपीटी दोनों
ai प्लेटफ़ॉर्म हैं जिनका उपयोग प्राकृतिक भाषा प्रोसेसिंग (एनएलपी) और मACHINE लर्निंग (एमएल) के लिए किया जाता है। दोनों प्लेटफ़ॉर्म के लिए उपयोग किया जाता है लेकिन उनके बीच कुछ मुख्य अंतर हैं।

डीपसीक एक ओपन-सोर्स प्लेटफ़ॉर्म है जिसका उपयोग प्राकृतिक भाषा प्रोसेसिंग और मACHINE लर्निंग के लिए किया जाता है। इसका उपयोग किसी भी प्रोग्रामिंग भाषा में किया जा सकता है और इसका सोर्स कोड ओपन-सोर्स है।

चैटजीपीटी एक क्लाउड-आधारित प्लेटफ़ॉर्म है जिसका उपयोग चैटबॉट्स और वोकेशनल मॉडल के लिए किया जाता है। इसका उपयोग प्रोग्रामिंग भाषा क्लाउडफॉरे में किया जाता है और इसका सोर्स कोड प्राइवेट है।

इन दोनों प्लेटफ़ॉर्म के बीच कुछ मुख्य अंतर हैं -

* डीपसीक ओपन-सोर्स है जबकि चैटजीपीटी प्राइवेट है।
* डीपसीक किसी भी प्रोग्रामिंग भाषा में किया जा सकता है जबकि चैटजीपीटी क्लाउडफॉरे में किया जाता है।
* डीपसीक का सोर्स कोड ओपन-सोर्स है जबकि चैटजीपीटी का सोर्स कोड प्राइवेट है।

इन अंतरों को देखते हुए डीपसीक और चैटजीपीटी के बीच कुछ मुख्य अंतर हैं। डीपसीक की ओपन

In [8]:
llm=ChatGroq(model='llama3-8b-8192')
llm=llm.bind_tools([tool])
result=llm.invoke("summarize the text on the webpage https://zapier.com/blog/deepseek-vs-chatgpt/ in Hindi")
result.tool_calls

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "deepseek-vs-chatgpt-summary"\n      },\n      "parameters": {\n        "lang": "hi",\n        "text": "https://zapier.com/blog/deepseek-vs-chatgpt/"\n      }\n    }\n  ]\n}\n</tool-use>'}}